<a href="https://colab.research.google.com/github/lukehartfield/Portfolio-Optimization/blob/main/LAS_Traders_Project_2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pillow

In [ ]:
!pip install wrds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [ ]:
!pip install adjustText

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
^C


In [ ]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import re
from datetime import datetime
from scipy.optimize import minimize
import math
import seaborn as sns
from pathlib import Path
from datetime import date
from calendar import monthrange

import wrds
db = wrds.Connection()


# -------------------------
# 1) Universe: all valid tickers (US common shares only)
# -------------------------

# NOTE: This query does not select a 'date' column, so don't pass date_cols=["date"].
q = """
    SELECT DISTINCT
        mse.ticker
    FROM crsp.msf AS msf
    JOIN crsp.msenames AS mse
      ON msf.permno = mse.permno
     AND mse.namedt <= msf.date
     AND (msf.date <= mse.nameendt OR mse.nameendt IS NULL)  -- handle open-ended names
    WHERE mse.shrcd IN (10, 11)          -- U.S. common shares only (no ADRs)
      AND mse.exchcd IN (1, 2, 3)        -- NYSE/AMEX/NASDAQ
"""
all_tickers = db.raw_sql(q)  # <-- no date_cols arg here
all_ticker_set = set(all_tickers["ticker"])

# -------------------------
# 2) Interactive ticker entry
# -------------------------

def get_tickers(all_ticker_set, prompt='Enter tickers separated by commas (e.g., AAPL, MSFT, TSLA) or quit to exit:'):
  """Accept commas as separators and returns clean tickers
  (uppercase/lowercase, letters/numbers, no duplicates, exist in the universe set)."""

  allow = re.compile(r"^[A-Z0-9]+$", re.IGNORECASE)   # alphanum only

  while True:
      raw = input(prompt)

      # Step 1: split, strip spaces, force uppercase
      tickers = [t.strip().upper() for t in raw.split(",") if t.strip()]

      if raw == "quit":
        print("See you next time.")
        return None

      # Step 2: validate
      not_found = [t for t in tickers if t not in all_ticker_set]

      # nothing inputted
      if not tickers:
          print("You must enter at least one ticker.")
          continue
      # tickers inputted but need to check if exist and correct
      seen = set()
      distinct_tickers = []
      invalid = []

      for t in tickers:
        if not allow.match(t):
          invalid.append(t)
          continue
        if t not in seen:
          seen.add(t)
          distinct_tickers.append(t)

      if invalid:
        print("Invalid ticker format (alphanumeric only): " + ", ".join(invalid))
        continue

      if not_found:
        print(f"Ticker(s) not found in WRDS: {', '.join(not_found)}. Try again.")
        continue

      return distinct_tickers

# -------------------------
# 3) Interactive portfolio weights
# -------------------------

# expects tickers to already be a list
def get_portfolio_weights(tickers,
                          prompt="What is your original portfolio weight for each ticker, in order, separated by commas? Enter 10 for 10%, equal for equal weights, or quit to exit\n",
                          min_w=-100.0, max_w=100.0, tolerance=1e-4): # ensures within weight constraint and prevents rounding error
  """Accept percent inputs and returns weights in decimal form, matching the
  number of tickers while also fitting the weight constraint."""

  # tickers = list(tickers)
  n = len(tickers)
  token_re = re.compile(r"^[+-]?[0-9]*\.?[0-9]+$")  # allow negatives like -10 or -0.5

  while True:
    raw = input(prompt).strip()
    user_w = [w for w in re.split(r"[,\s]+", raw) if w]

    if raw == "quit":
      print("See you next time.")
      break

    # need equal option
    if raw.lower() == "equal":
      eq = [1.0 / n] * n
      return pd.Series(eq, index=tickers)

    # n weights dont match n tickers
    if len(user_w) != n:
      print(f"Please provide exactly {n} weights to match: {', '.join(tickers)}")
      continue

    bad = [w for w in user_w if not token_re.match(w)]
    if bad:
      print("Invalid token(s). Use percent numbers only (e.g., 10, 1.2, -5). Offenders: " + ", ".join(bad))
      continue

    if not raw:
      print("Please enter your portfolio weights.")

    w_pct = [float(w) for w in user_w]

    # per-asset bounds
    below = [i for i, v in enumerate(w_pct) if v < min_w]
    above = [i for i, v in enumerate(w_pct) if v > max_w]
    if below or above:
      if below:
        print("Below minimum bound "
              f"({min_w}%): " + ", ".join(f"{tickers[i]}={w_pct[i]}%" for i in below))
      if above:
        print("Above maximum bound "
              f"({max_w}%): " + ", ".join(f"{tickers[i]}={w_pct[i]}%" for i in above))
      continue

    total = sum(w_pct)
    if abs(total - 100.0) > tolerance:
       print(f"Total must be 100%. Your total is {total:.4f}%. Please adjust and try again.")
       continue

    return pd.Series([v/100.0 for v in w_pct], index=tickers)



# -------------------------
# 4) Interactive date entry
# -------------------------

def get_dates():
  """Prompt for YYYY-MM, snap to month boundaries, validate order,
  and ensure OOS starts after IS ends. Returns four YYYY-MM-DD strings."""
  ym_pat = re.compile(r"^\d{4}-(0[1-9]|1[0-2])$")  # strict YYYY-MM

  while True:
    is_start_in = input("In-sample start (YYYY-MM): ").strip()
    is_end_in   = input("In-sample end   (YYYY-MM): ").strip()
    oos_start_in= input("Out-of-sample start (YYYY-MM): ").strip()
    oos_end_in  = input("Out-of-sample end   (YYYY-MM): ").strip()

    # exit, only one of these will show
    if is_start_in.lower() in {"q", "quit"}: return print("See you next time.")
    if is_end_in.lower() in {"q", "quit"}: return print("See you next time.")
    if oos_start_in.lower() in {"q", "quit"}: return print("See you next time.")
    if oos_end_in.lower() in {"q", "quit"}: return print("See you next time.")

    # presence
    if not is_start_in or not is_end_in or not oos_start_in or not oos_end_in:
        print("All four dates are required in YYYY-MM. Try again.")
        continue

    # format
    if not (ym_pat.match(is_start_in) and ym_pat.match(is_end_in)
            and ym_pat.match(oos_start_in) and ym_pat.match(oos_end_in)):
        print("Incorrect format. Use YYYY-MM for each date. Try again.")
        continue

    # parse and snap to boundaries
    y,m = map(int, is_start_in.split("-"));  is_start = date(y, m, 1)
    y,m = map(int, is_end_in.split("-"));    is_end   = date(y, m, monthrange(y, m)[1])
    y,m = map(int, oos_start_in.split("-")); oos_start= date(y, m, 1)
    y,m = map(int, oos_end_in.split("-"));   oos_end  = date(y, m, monthrange(y, m)[1])

    # ordering checks: OOS dates need to be after IS dates
    if is_start > is_end:
        print(f"In-sample start {is_start} must be <= end {is_end}. Try again.")
        continue
    if oos_start > oos_end:
        print(f"Out-of-sample start {oos_start} must be <= end {oos_end}. Try again.")
        continue
    if not (oos_start > is_end):
        print(f"OOS must start after IS ends. IS end {is_end}, OOS start {oos_start}. Try again.")
        continue

# guard against out-of-reach OOS dates
    try:
      maxd = db.raw_sql("SELECT MAX(date) AS maxd FROM crsp.msf")["maxd"].iloc[0]  # noqa: F821
      if pd.notna(maxd):
        maxd = pd.to_datetime(maxd).date()
        if oos_start > maxd:
          print(f"OOS start {oos_start} is after latest CRSP month {maxd}. Pick earlier dates."); continue
        if oos_end > maxd:
          print(f"Clamping OOS end from {oos_end} to {maxd} (latest available).")
          oos_end = maxd
    except Exception:
        pass

    return tuple(d.strftime("%Y-%m-%d") for d in (is_start, is_end, oos_start, oos_end))


# -------------------------
# 5) CRSP fetch helper
# -------------------------

# we felt a minimum observation of 24 seemed good because <2yrs is too noisy

def fetch_crsp_returns(db, tickers, start_ymd, end_ymd, min_obs=24, permnos=None):
    """ Pull monthly CRSP for tickers and date range, clean + combine delisting returns,
    enforce coverage, and create a monthly return matrix (ret_total).
    Returns ret_matrix, kept and dropped tickers."""
    if not tickers and not permnos:
        raise ValueError("No tickers provided.")

    # CHANGE 2: build the ID filter once
    if permnos is not None:
      if len(permnos) == 0:
            raise ValueError("permnos provided but empty.")
      id_filter_sql = "msf.permno IN (" + ",".join(str(int(p)) for p in permnos) + ")"
    else:
        tickers_sql = ",".join("'" + t.replace("'", "''") + "'" for t in tickers)
        id_filter_sql = f"n.ticker IN ({tickers_sql})"

    q = f"""
        SELECT
            msf.permno,
            msf.date,
            msf.ret,
            msf.shrout,
            msf.prc,
            n.shrcd,
            n.exchcd,
            n.ticker,
            dl.dlret
        FROM crsp.msf AS msf
        JOIN crsp.msenames AS n
          ON msf.permno = n.permno
         AND n.namedt <= msf.date
         AND (msf.date <= n.nameendt OR n.nameendt IS NULL)
        LEFT JOIN crsp.msedelist AS dl
          ON msf.permno = dl.permno
         AND date_trunc('month', dl.dlstdt) = date_trunc('month', msf.date)
        WHERE msf.date BETWEEN '{start_ymd}' AND '{end_ymd}'
          AND n.shrcd IN (10, 11, 12)     -- US common shares/close equivalents
          AND n.exchcd IN (1, 2, 3)       -- NYSE/AMEX/NASDAQ
          AND {id_filter_sql}             -- CHANGE 3: use unified filter

        ORDER BY msf.permno, msf.date;
    """

    df = db.raw_sql(q, date_cols=["date"])
    if df.empty:
        raise RuntimeError("CRSP query returned no rows. Check tickers and dates.")

    # cleanup
    df["prc"] = df["prc"].abs()
    df["ret"] = pd.to_numeric(df["ret"], errors="coerce") # sets to NaN instead of error
    df["dlret"] = pd.to_numeric(df["dlret"], errors="coerce").fillna(0.0)

    # incl delisting return: ret_total = (1+ret)*(1+dlret) - 1
    df["ret_total"] = (1.0 + df["ret"]) * (1.0 + df["dlret"]) - 1.0

    # Month-end key for pivot
    df["month"] = pd.to_datetime(df["date"]).dt.to_period("M").dt.to_timestamp("M")

    # calc Market cap
    df["shrout"] = pd.to_numeric(df["shrout"], errors="coerce")
    df["shrout_shares"] = df["shrout"] * 1000.0
    df["me"] = df["prc"] * df["shrout_shares"]

    # drop rows w/missing returns
    df = df.dropna(subset=["ret_total"])

    # Deduplicate (permno, month)
    df = df.sort_values(["permno", "month"])
    df = df.drop_duplicates(subset=["permno", "month"], keep="last")

    # coverage - keep tickers with at least min_obs non-null returns
    counts = df.groupby("ticker")["ret_total"].apply(lambda s: s.notna().sum())
    kept = counts[counts >= min_obs].index.tolist()
    dropped = sorted(set(df["ticker"]) - set(kept))

    if not kept:
        raise RuntimeError(f"All tickers dropped by coverage (min_obs={min_obs}).")

    df = df[df["ticker"].isin(kept)].copy()

    # Pivot to return matrix (rows=month-end, cols=tickers)
    ret_matrix = (
        df.pivot_table(index="month", columns="ticker", values="ret_total", aggfunc="last")
          .sort_index()
    )
    ret_matrix.index = pd.to_datetime(ret_matrix.index).to_period("M").to_timestamp("M")

    return ret_matrix, kept, dropped, df # need df later

from pathlib import Path
import pandas as pd

def build_is_oos_matrices(
    db,
    tickers,
    is_start, is_end,
    oos_start, oos_end,
    *,
    min_obs_is: int = 24,
    min_obs_oos: int = 1,
    save_csv: bool = True,
    outdir: str | Path = "outputs",
    verbose: bool = True,
):
    """
    Build IS/OOS return matrices aligned by PERMNO (robust to ticker drift),
    then relabel columns to IS tickers (last IS label per permno), clean zero-variance,
    realign OOS to IS columns/order, and optionally save CSVs.

    Returns a dict with:
      - is_ret_matrix : IS matrix (rows=month_end, cols=tickers; cleaned)
      - os_ret_matrix : OOS matrix (aligned to IS columns)
      - permno_to_ticker : dict[int -> str] mapping (from IS, last label per permno)
      - ret_perm : IS matrix by permno (before relabel/clean)
      - oos_perm : OOS matrix by permno (aligned to IS permnos)
      - df_is, df_oos : cleaned long frames from fetches
      - kept_is, dropped_is, kept_oos, dropped_oos : lists
    """
    # --- In-sample ---
    ret_matrix, kept_is, dropped_is, df_is = fetch_crsp_returns(
        db, tickers, is_start, is_end, min_obs=min_obs_is
    )
    if verbose:
        print("IS matrix (raw):", ret_matrix.shape, "| kept:", kept_is, "| dropped:", dropped_is)

    # Resolve IS permnos from cleaned IS df
    permnos_is = sorted(df_is["permno"].dropna().astype(int).unique().tolist())
    if verbose:
        print("Resolved PERMNOs (IS):", permnos_is)

    # --- Out-of-sample (same permno universe; minimal coverage) ---
    os_matrix, kept_oos, dropped_oos, df_oos = fetch_crsp_returns(
        db, kept_is, oos_start, oos_end, min_obs=min_obs_oos, permnos=permnos_is
    )

    # --- Build permno-based matrices (robust to ticker string drift) ---
    ret_perm = (
        df_is.pivot_table(index="month", columns="permno", values="ret_total", aggfunc="last")
            .sort_index()
    )
    oos_perm = (
        df_oos.pivot_table(index="month", columns="permno", values="ret_total", aggfunc="last")
            .sort_index()
            .reindex(columns=ret_perm.columns)  # align OOS to IS permnos
    )

    # --- Map permno -> IS ticker (last label per permno in IS window) ---
    permno_to_ticker = (
        df_is.sort_values(["permno", "month"])
             .drop_duplicates(subset=["permno"], keep="last")
             .set_index("permno")["ticker"].astype(str).to_dict()
    )

    # Relabel columns for presentation
    ret_matrix_tickers = ret_perm.rename(columns=permno_to_ticker)
    os_ret_matrix_lbl  = oos_perm.rename(columns=permno_to_ticker)

    if verbose:
        print("IS (tickers):", ret_matrix_tickers.shape)
        print("OOS (tickers):", os_ret_matrix_lbl.shape)

    # --- Clean IS: drop all-NaN, then zero-variance columns ---
    ret_is = ret_matrix_tickers.dropna(axis=1, how="all").copy()
    std_is = ret_is.std(skipna=True)
    keep_cols = std_is[std_is > 0].index.tolist()
    ret_is = ret_is[keep_cols]

    # --- Realign OOS to cleaned IS columns/order ---
    ret_oos = os_ret_matrix_lbl.reindex(columns=ret_is.columns)

    # Name index and optionally save
    ret_is.index.name  = "month_end"
    ret_oos.index.name = "month_end"

    if save_csv:
        outdir = Path(outdir); outdir.mkdir(parents=True, exist_ok=True)
        ret_is.to_csv(outdir / "IS_ret_matrix.csv",  float_format="%.8f", na_rep="NA")
        ret_oos.to_csv(outdir / "OOS_ret_matrix.csv", float_format="%.8f", na_rep="NA")
        if verbose:
            print("Saved CSVs to:", outdir)

    return {
        "is_ret_matrix": ret_is,
        "os_ret_matrix": ret_oos,
        "permno_to_ticker": permno_to_ticker,
        "ret_perm": ret_perm,
        "oos_perm": oos_perm,
        "df_is": df_is,
        "df_oos": df_oos,
        "kept_is": kept_is,
        "dropped_is": dropped_is,
        "kept_oos": kept_oos,
        "dropped_oos": dropped_oos,
    }

# get avg for each stock to demean later
def get_mean(ret_matrix: pd.DataFrame) -> pd.Series:
    """
    Input:  ret_matrix = DataFrame of monthly simple returns (60 rows x N tickers).
    Output: Series of average return per ticker (NaNs ignored).
    """
    # If there are non-numeric cols, coerce them (optional; remove if not needed)
    rets_num = ret_matrix.apply(pd.to_numeric, errors="coerce")
    return rets_num.mean(axis=0)  # monthly mean per column

# monthly rawret - mean = monthly emat
def demean_returns(ret_matrix: pd.DataFrame, means: pd.Series | None = None) -> pd.DataFrame:
    """
    Input:  rets  = DataFrame of monthly returns (same shape as above)
            means = (optional) Series of per-ticker means; if None, compute from rets
    Output: DataFrame where each column has its mean subtracted (demeaned), also know as
            the emat.
    """
    rets_num = ret_matrix.apply(pd.to_numeric, errors="coerce")
    if means is None:
        means = rets_num.mean(axis=0)
    # Broadcast subtract per column; NaNs stay NaN
    return rets_num.subtract(means, axis=1)

def create_covmat(emat: pd.DataFrame) -> pd.DataFrame:
    """
    E: DataFrame of demeaned returns (rows = time, cols = tickers), no NaNs.
    Returns the sample covariance matrix using (E' E) / (n - 1).
    """
    # ensure complete cases (Excel usually assumes same n per column here)
    # maybe return error if not complete?
    E = emat.dropna(axis=0, how="any")
    n = E.shape[0]
    cov_np = (E.T @ E).to_numpy() / (n - 1)
    return pd.DataFrame(cov_np, index=E.columns, columns=E.columns)

def minverse(covmat: pd.DataFrame) -> pd.DataFrame:
    """Excel-like MINVERSE: returns (covmat)^(-1) as a DataFrame."""
    A = covmat.to_numpy(dtype=float)        # ensure float
    I = np.eye(A.shape[0], dtype=float)
    invA = np.linalg.solve(A, I)            # solves A * X = I => X = A' * I
    return pd.DataFrame(invA, index=covmat.index, columns=covmat.columns)


def get_N_T(emat: pd.DataFrame) -> tuple[int, int]:
    """
    N = number of tickers (columns), T = number of rows (after complete-case drop)
    """
    E = emat.dropna(axis=0, how="any")
    T = E.shape[0]
    N = E.shape[1]
    return N, T

def get_variances(emat: pd.DataFrame) -> pd.Series:
    """
    Column-wise sample variances of demeaned returns.
    Assumes emat is already demeaned (mean ~ 0 per column).
    """
    E = emat.dropna(axis=0, how="any")
    return E.var(axis=0, ddof=1)

def avg_variance(emat: pd.DataFrame) -> float:
    """
    Average of column variances (i.e., trace(S)/N).
    """
    return float(get_variances(emat).mean())

def port_stdev(w, c):
  return (float(w.T @ c @ w))**0.5

def create_betas(emat: pd.DataFrame, covmat: pd.DataFrame) -> pd.Series:
    """
    For each row t of emat, compute:
      SSE_t = sum( ( x_t x_t' - covmat )^2 )
    where x_t is the 1×N demeaned return vector at time t.
    Returns a Series indexed like emat.index.
    """
    E = emat.dropna(axis=0, how="any").to_numpy(dtype=float)  # T×N
    S = covmat.reindex(index=emat.columns, columns=emat.columns).to_numpy(dtype=float)  # N×N

    sse_list = []
    for x in E:  # x is shape (N,)
        outer = np.outer(x, x)        # N×N
        diff = outer - S
        sse_list.append(float(np.sum(diff * diff)))  # Frobenius norm squared

    return pd.Series(sse_list, index=emat.dropna(axis=0, how="any").index, name="row_sse")

def create_ident(N: int, avg_var: float, columns: list[str] | None = None) -> pd.DataFrame:
    I = np.eye(N) * avg_var
    if columns is None:
        return pd.DataFrame(I)
    return pd.DataFrame(I, index=columns, columns=columns)

def delta_squared(covmat: pd.DataFrame, ident: pd.DataFrame) -> float:
    """
    Returns SUM((covmat - ident)^2) with index/column alignment.
    """
    # align to common order (and error if mismatch)
    cov = covmat.copy()
    idt = ident.reindex(index=cov.index, columns=cov.columns)
    if idt.isnull().values.any():
        raise ValueError("ident must have the same index/columns as covmat.")

    diff = cov.to_numpy(dtype=float) - idt.to_numpy(dtype=float)
    return float(np.sum(diff * diff))  # Frobenius norm squared

def beta_squared(betas, T: int) -> float:
    if T <= 0:
        raise ValueError("T must be > 0.")
    return float(np.nansum(np.asarray(betas, float))) / (T**2)

def get_lambda(emat: pd.DataFrame, covmat: pd.DataFrame) -> float:
    """
    λ = clip( β² / δ², 0, 1 ), with β² = mean row SSEs from create_betas.
    """
    ident = create_ident(N, avg_var, columns=emat.columns)
    betas = create_betas(emat, covmat)
    beta2 = beta_squared(betas,T)
    delta2 = delta_squared(covmat, ident)
    if delta2 <= 0:
        return 0.0
    lam = beta2 / delta2
    return float(min(1.0, max(0.0, lam)))

def calc_lw_covmat(covmat: pd.DataFrame, ident: pd.DataFrame, lam: float) -> pd.DataFrame:

    # Align labels
    ident_aligned = ident.reindex(index=covmat.index, columns=covmat.columns)
    if ident_aligned.isnull().values.any():
        raise ValueError("ident must have the same index/columns as covmat.")

    return lam * ident_aligned + (1.0 - lam) * covmat

def portfolio_return(weights: pd.Series, means: pd.Series) -> float:
    """
    Computes MMULT(TRANSPOSE(weights), means).
    Both inputs indexed by the same tickers.
    Returns a scalar (same period as `means`).
    """
    w = pd.to_numeric(weights, errors="coerce")
    m = pd.to_numeric(means, errors="coerce")
    m = m.reindex(w.index)   # align by ticker names
    return float(np.dot(w.values, m.values))

def closedformgmv(invcovmat: pd.DataFrame) -> pd.Series:
    """
    Global Minimum Variance (GMV) weights given INVCOVMAT.
    Numerator:   INVCOVMAT * ONEVEC
    Denominator: 1' * INVCOVMAT * 1
    """
    Ainv = invcovmat.to_numpy(dtype=float)
    n = Ainv.shape[0]
    one = np.ones((n, 1))
    num = Ainv @ one
    den = float(one.T @ Ainv @ one)
    w = (num / den).ravel()
    return pd.Series(w, index=invcovmat.index)  # sums to ~1

def calc_efficientportfolio(invcovmat: pd.DataFrame, means: pd.Series, mu0: float) -> pd.Series:
    """
    Unconstrained efficient-portfolio weights for target return mu0.
    Uses: w = P[(c - b*mu0)/d * 1 + (a*mu0 - b)/d * mu],
    where P = invcovmat, a=1'P1, b=1'Pmu, c=mu'Pmu, d=ac-b^2.
    """
    P   = invcovmat.to_numpy(dtype=float)
    mu  = means.reindex(invcovmat.index).to_numpy(dtype=float).reshape(-1, 1)
    one = np.ones((P.shape[0], 1))

    a = float(one.T @ P @ one)
    b = float(one.T @ P @ mu)
    c = float(mu.T  @ P @ mu)
    d = a * c - b * b

    w = ((c - b*mu0)/d) * (P @ one) + ((a*mu0 - b)/d) * (P @ mu)
    w = w.ravel()
    w = w / w.sum()  # tidy sum-to-one

    return pd.Series(w, index=invcovmat.index, name="Eff_Weight")


def calc_orp(invcovmat: pd.DataFrame, means: pd.Series, rfrate: float) -> pd.Series:
      """
      Optimal Risky Portfolio (tangency) weights.
      Excel analog:
        Numerator   = MMULT(INVCOVMAT, muvec - rfrate)
        Denominator = MMULT(TRANSPOSE(ONEVEC), MMULT(INVCOVMAT, muvec - rfrate))
      Returns weights that sum to 1.
      """
      # Align and cast
      mu = means.reindex(invcovmat.index).to_numpy(dtype=float).reshape(-1, 1)
      P  = invcovmat.to_numpy(dtype=float)
      e  = np.ones((P.shape[0], 1))

      x = P @ (mu - rfrate * e)            # numerator vector
      denom = float(e.T @ x)               # scalar denominator
      w = (x / denom).ravel()              # normalize to sum to 1
      return pd.Series(w, index=invcovmat.index, name="ORP_Weight")

def calc_orp_scipy(covmat: pd.DataFrame, means: pd.Series, rfrate: float,
                   lb: float = -1e6, ub: float = 1e6, ftol: float = 1e-9, maxiter: int = 2000) -> pd.Series:
    """
    Tangency (ORP) via SLSQP with box bounds lb <= w_i <= ub and sum(w)=1.
    Maximizes Sharpe: (w·mu - rf) / sqrt(w'Σw).
    """
    # Align inputs
    assert list(covmat.index) == list(covmat.columns), "covmat index/columns mismatch"
    mu = means.reindex(covmat.index).to_numpy(dtype=float)
    Σ  = covmat.to_numpy(dtype=float)
    n  = len(mu)

    # Feasibility: sum(w)=1 with box bounds
    if lb > ub:
        raise ValueError("lb > ub")
    if n*lb - 1.0 > 1e-12 or 1.0 - n*ub > 1e-12:
        raise ValueError(f"Infeasible bounds for sum=1: need n*lb <= 1 <= n*ub (n={n}, lb={lb}, ub={ub}).")

    # Objective (minimize negative Sharpe)
    def neg_sharpe(w: np.ndarray) -> float:
        pr = float(w @ mu)
        v2 = float(w @ Σ @ w)
        if v2 <= 0:
            return 1e12
        return -(pr - rfrate) / (v2 ** 0.5)

    cons   = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},)
    bounds = [(lb, ub)] * n
    w0     = np.full(n, 1.0 / n)

    res = minimize(neg_sharpe, w0, method='SLSQP', bounds=bounds, constraints=cons,
                   options={'ftol': ftol, 'maxiter': maxiter})
    if not res.success:
        raise RuntimeError(f"Optimization failed: {res.message}")
    return pd.Series(res.x, index=covmat.index, name="ORP_Weight")


# for excel
def save_current_fig(path):
    import matplotlib.pyplot as plt
    plt.tight_layout()
    plt.savefig(path, dpi=200, bbox_inches="tight")
    print(f"[Plot] Saved {path}")



In [ ]:
#Call Functions

#Get user inputs
dates = get_dates()
is_start  = dates[0]
is_end    = dates[1]
oos_start = dates[2]
oos_end   = dates[3]
tickers = get_tickers(all_ticker_set)
weights = get_portfolio_weights(tickers)

#Build returns
res = build_is_oos_matrices(db, tickers,
                            is_start, is_end, oos_start, oos_end,
                            min_obs_is=24, min_obs_oos=1, save_csv=True)

ret_matrix = res["is_ret_matrix"]
os_ret_matrix = res["os_ret_matrix"]


In-sample start (YYYY-MM): 2014-01
In-sample end   (YYYY-MM): 2020-12
Out-of-sample start (YYYY-MM): 2021-01
Out-of-sample end   (YYYY-MM): 2023-12
Enter tickers separated by commas (e.g., AAPL, MSFT, TSLA) or quit to exit:DE, LMT, DIS, MCD, WMT, NKE, LUV, AMD, AMZN, TSLA
What is your original portfolio weight for each ticker, in order, separated by commas? Enter 10 for 10%, equal for equal weights, or quit to exit
equal
IS matrix (raw): (84, 10) | kept: ['AMD', 'AMZN', 'DE', 'DIS', 'LMT', 'LUV', 'MCD', 'NKE', 'TSLA', 'WMT'] | dropped: []
Resolved PERMNOs (IS): [19350, 21178, 26403, 43449, 55976, 57665, 58683, 61241, 84788, 93436]
IS (tickers): (84, 10)
OOS (tickers): (36, 10)
Saved CSVs to: outputs


In [ ]:
#Get means
means = get_mean(ret_matrix)
os_means = get_mean(os_ret_matrix)

#Get Emats
emat = demean_returns(ret_matrix,means)
os_emat = demean_returns(os_ret_matrix,os_means)

#Create Covmats
covmat = create_covmat(emat)
os_covmat = create_covmat(os_emat)

invcovmat = minverse(covmat)

#Get Ledoit Wolf Features
N,T = get_N_T(emat)
betas = create_betas(emat, covmat)
avg_var = avg_variance(emat)
ident = create_ident(N, avg_var, columns=emat.columns)
lam = get_lambda(emat, covmat)
lw_covmat = calc_lw_covmat(covmat, ident, lam)
os_lw_covmat = calc_lw_covmat(os_covmat, ident, lam)
lwinvcovmat = minverse(lw_covmat)


In [ ]:
rfrate = .035/12
mu0 = portfolio_return(weights,means)

#Get GMVs
gmv = closedformgmv(invcovmat)
lwgmv = closedformgmv(lwinvcovmat)

#Get Efficient Portfolios
efport = calc_efficientportfolio(invcovmat, means, mu0)
lwefport = calc_efficientportfolio(lwinvcovmat,means,mu0)

#Get Regular ORP
orp = calc_orp(invcovmat,means,rfrate)
lworp = calc_orp(lwinvcovmat,means,rfrate)

#Get quadratic programming orp
qporp = calc_orp_scipy(covmat, means, rfrate, lb=-1.0, ub=1.0)
lwqporp = calc_orp_scipy(lw_covmat, means, rfrate, lb=-1.0, ub=1.0)


/tmp/ipython-input-4073525548.py:573: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  den = float(one.T @ Ainv @ one)
/tmp/ipython-input-4073525548.py:587: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = float(one.T @ P @ one)
/tmp/ipython-input-4073525548.py:588: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b = float(one.T @ P @ mu)
/tmp/ipython-input-4073525548.py:589: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will e

In [ ]:
print("\n=== In-Sample Portfolio Weights (regular Σ) ===")
print("Original Portfolio Weights:")
print(weights, "\n")

print("GMV Weights:")
print(gmv, "\n")

print("Efficient Portfolio Weights:")
print(efport, "\n")

print("ORP Weights:")
print(orp, "\n")

print("QP ORP Weights:")
print(qporp, "\n")


print("\n=== In-Sample Portfolio Weights (Ledoit–Wolf Σ) ===")
print("LW GMV Weights:")
print(lwgmv, "\n")

print("LW Efficient Portfolio Weights:")
print(lwefport, "\n")

print("LW ORP Weights:")
print(lworp, "\n")

print("LW QP ORP Weights:")
print(lwqporp, "\n")



=== In-Sample Portfolio Weights (regular Σ) ===
Original Portfolio Weights:
DE      0.1
LMT     0.1
DIS     0.1
MCD     0.1
WMT     0.1
NKE     0.1
LUV     0.1
AMD     0.1
AMZN    0.1
TSLA    0.1
dtype: float64 

GMV Weights:
permno
DE      0.086199
LMT     0.172532
DIS     0.003709
MCD     0.315040
WMT     0.294299
NKE     0.145442
LUV    -0.007563
AMD    -0.034402
AMZN    0.039580
TSLA   -0.014837
dtype: float64 

Efficient Portfolio Weights:
permno
DE      0.073005
LMT     0.124374
DIS    -0.163064
MCD     0.218156
WMT     0.147775
NKE     0.270910
LUV     0.004357
AMD     0.082534
AMZN    0.149984
TSLA    0.091968
Name: Eff_Weight, dtype: float64 

ORP Weights:
permno
DE      0.066097
LMT     0.099163
DIS    -0.250371
MCD     0.167437
WMT     0.071069
NKE     0.336593
LUV     0.010597
AMD     0.143751
AMZN    0.207782
TSLA    0.147881
Name: ORP_Weight, dtype: float64 

QP ORP Weights:
permno
DE      0.066092
LMT     0.099172
DIS    -0.250380
MCD     0.167452
WMT     0.071093
NKE  

In [ ]:

#Build Returns
pfs = pd.DataFrame(columns=["Expected Return","Standard Deviation"])
pfs.loc['IS Orig Port'] = [portfolio_return(weights, means), port_stdev(weights, covmat)]
pfs.loc['IS GMV'] = [portfolio_return(gmv, means), port_stdev(gmv, covmat)]
pfs.loc['IS Efficient Portfolio'] = [portfolio_return(efport, means), port_stdev(efport, covmat)]
pfs.loc['IS ORP'] = [portfolio_return(orp, means), port_stdev(orp, covmat)]
pfs.loc['IS QP ORP'] = [portfolio_return(qporp, means), port_stdev(qporp, covmat)]

pfs.loc['OS Orig Port']  = [portfolio_return(weights, os_means), port_stdev(weights, os_covmat)]
pfs.loc['OS GMV'] = [portfolio_return(gmv, os_means), port_stdev(gmv, os_covmat)]
pfs.loc['OS Efficient Portfolio'] = [portfolio_return(efport, os_means), port_stdev(efport, os_covmat)]
pfs.loc['OS ORP'] = [portfolio_return(orp, os_means), port_stdev(orp, os_covmat)]
pfs.loc['OS QP ORP'] = [portfolio_return(qporp, os_means), port_stdev(qporp, os_covmat)]

pfs['Sharpe Ratio (Monthly)'] = (pfs['Expected Return']-rfrate)/pfs['Standard Deviation']
pfs['Sharpe Ratio (Annualized)'] = (pfs['Sharpe Ratio (Monthly)'])*(12**0.5)

# pfs['Expected Return (Annualized)'] = pfs['Expected Return'] * 12
# pfs['Standard Deviation (Annualized)'] = pfs['Standard Deviation'] * (12 ** 0.5)

pfslw = pd.DataFrame(columns=["Expected Return","Standard Deviation"])
pfslw.loc['IS LW Orig Port'] = [portfolio_return(weights,    means),    port_stdev(weights,    lw_covmat)]
pfslw.loc['IS LW GMV']       = [portfolio_return(lwgmv,     means),    port_stdev(lwgmv,       lw_covmat)]
pfslw.loc['IS LW Efficient'] = [portfolio_return(lwefport,  means),    port_stdev(lwefport,    lw_covmat)]
pfslw.loc['IS LW ORP']       = [portfolio_return(lworp,     means),    port_stdev(lworp,       lw_covmat)]
pfslw.loc['IS LW QP ORP']    = [portfolio_return(lwqporp,   means),    port_stdev(lwqporp,     lw_covmat)]

pfslw.loc['OS LW Orig Port'] = [portfolio_return(weights,    os_means),    port_stdev(weights,    os_lw_covmat)]
pfslw.loc['OS LW GMV']       = [portfolio_return(lwgmv,     os_means),    port_stdev(lwgmv,       os_lw_covmat)]
pfslw.loc['OS LW Efficient'] = [portfolio_return(lwefport,  os_means),    port_stdev(lwefport,    os_lw_covmat)]
pfslw.loc['OS LW ORP']       = [portfolio_return(lworp,     os_means),    port_stdev(lworp,       os_lw_covmat)]
pfslw.loc['OS LW QP ORP']    = [portfolio_return(lwqporp,   os_means),    port_stdev(lwqporp,     os_lw_covmat)]

pfslw['Sharpe Ratio (Monthly)'] = (pfslw['Expected Return']-rfrate)/pfslw['Standard Deviation']
pfslw['Sharpe Ratio (Annualized)'] = (pfslw['Sharpe Ratio (Monthly)'])*(12**0.5)

# pfslw['Expected Return (Annualized)'] = pfs['Expected Return'] * 12
# pfslw['Standard Deviation (Annualized)'] = pfs['Standard Deviation'] * (12 ** 0.5)

print(pfs)
print(pfslw)

#Call Graphs




                        Expected Return  Standard Deviation  \
IS Orig Port                   0.023423            0.052088   
IS GMV                         0.012308            0.035956   
IS Efficient Portfolio         0.023423            0.047928   
IS ORP                         0.029241            0.060198   
IS QP ORP                      0.029241            0.060198   
OS Orig Port                   0.006700            0.062813   
OS GMV                         0.007333            0.047159   
OS Efficient Portfolio         0.011918            0.058834   
OS ORP                         0.014318            0.074441   
OS QP ORP                      0.014319            0.074440   

                        Sharpe Ratio (Monthly)  Sharpe Ratio (Annualized)  
IS Orig Port                          0.393682                   1.363754  
IS GMV                                0.261200                   0.904823  
IS Efficient Portfolio                0.427854                   1.482129  
IS

In [ ]:

from matplotlib.lines import Line2D
import matplotlib.patheffects as pe

HALO = [pe.withStroke(linewidth=3, foreground="white")]  # outline around text

def efficient_frontier_unconstrained(means: pd.Series, covmat: pd.DataFrame, n: int = 300):
    idx = covmat.index
    mu  = means.reindex(idx).to_numpy(dtype=float).reshape(-1, 1)
    P   = np.linalg.inv(covmat.to_numpy(dtype=float))
    one = np.ones_like(mu)
    a = float(one.T @ P @ one); b = float(one.T @ P @ mu); c = float(mu.T @ P @ mu)
    d = a * c - b * b
    mu_gmv = b / a
    mu_hi  = max(float(means.max()), mu_gmv) * 1.2
    mus = np.linspace(mu_gmv, mu_hi, n)
    sig2 = (a*mus**2 - 2*b*mus + c) / d
    sig  = np.sqrt(np.maximum(sig2, 0.0))
    return sig, mus

def _letter_map_from_index(index_like) -> dict:
    # A, B, C, ... Z, AA, AB, ...
    letters, k = [], 0
    for _ in range(len(index_like)):
        n, s = k, ""
        while True:
            s = chr(ord('A') + (n % 26)) + s
            n = n // 26 - 1
            if n < 0: break
        letters.append(s); k += 1
    return {name: lab for name, lab in zip(index_like, letters)}

def _add_letter_legend(ax, mapping, title="Letter → Portfolio", loc="lower right"):
    labels = [f"{lab}: {name}" for name, lab in mapping.items()]
    handles = [Line2D([0],[0], color='none') for _ in labels]  # invisible handles
    leg = ax.legend(handles, labels, title=title, frameon=True, loc=loc)
    ax.add_artist(leg)

def _annotate_static(ax, df, mask, color, letter_map, dy_pixels):
    # Fixed offset labels, no arrows, with a white bubble + halo to stand out
    for name, row in df.loc[mask].iterrows():
        x, y = row['Standard Deviation'], row['Expected Return']
        if not (np.isfinite(x) and np.isfinite(y)):
            continue
        label = letter_map.get(name, name)
        ax.annotate(
            label, (x, y),
            xytext=(0, dy_pixels), textcoords="offset points",
            ha="center", va=("bottom" if dy_pixels > 0 else "top"),
            fontsize=10, fontweight="bold", color=color,
            bbox=dict(boxstyle="round,pad=0.25", fc="white", ec="none", alpha=0.9),
            path_effects=HALO, zorder=6, clip_on=False
        )

def plot_is_os_frontier(
    pfs: pd.DataFrame, means: pd.Series, covmat: pd.DataFrame, rfrate: float, title: str,
    color_is='#1f77b4', color_os='#ff7f0e', cal_color='#2ca02c', frontier_color='#17becf',
    use_letters: bool = True, letter_legend_loc: str = "lower right"
):
    # numeric guard
    for col in ['Standard Deviation', 'Expected Return']:
        pfs[col] = pd.to_numeric(pfs[col], errors='coerce')

    idx_norm = pfs.index.astype(str).str.lower().str.strip()
    is_mask = idx_norm.str.startswith('is ')
    os_mask = idx_norm.str.startswith('os ')
    letter_map = _letter_map_from_index(pfs.index.tolist()) if use_letters else {}

    sns.set(style='whitegrid')
    fig, ax = plt.subplots(figsize=(8.5,6.5), dpi=160)

    # Points with white edge for contrast
    if pfs.loc[is_mask].size:
        sub = pfs.loc[is_mask, ['Standard Deviation','Expected Return']].dropna()
        ax.scatter(sub['Standard Deviation'], sub['Expected Return'],
                   c=color_is, s=110, label='IS', zorder=4,
                   edgecolors="white", linewidths=1.2)
        _annotate_static(ax, pfs, is_mask, color_is, letter_map, dy_pixels=12)

    if pfs.loc[os_mask].size:
        sub = pfs.loc[os_mask, ['Standard Deviation','Expected Return']].dropna()
        ax.scatter(sub['Standard Deviation'], sub['Expected Return'],
                   c=color_os, s=130, marker='^', label='OOS', zorder=4,
                   edgecolors="white", linewidths=1.2)
        _annotate_static(ax, pfs, os_mask, color_os, letter_map, dy_pixels=-14)

    # CAL via "IS QP ORP" (full line kept)
    tag = 'is qp orp'
    poss = idx_norm[idx_norm.str.contains(tag)].tolist()
    if poss:
        label = pfs.index[idx_norm == poss[0]][0]
        sd = float(pfs.loc[label, 'Standard Deviation']); er = float(pfs.loc[label, 'Expected Return'])
        if np.isfinite(sd) and np.isfinite(er):
            slope = (er - rfrate)/max(sd, 1e-12)
            x_max = max(float(pfs['Standard Deviation'].max())*1.25, sd*1.35)
            xs = np.linspace(0, x_max, 400); ys = rfrate + slope*xs
            ax.plot(xs, ys, color=cal_color, linestyle='--', linewidth=2.4,
                    label=f'CAL via {label}', zorder=2.5)
            ax.scatter([0],[rfrate], color=cal_color, edgecolor='k', zorder=5)

    # Efficient frontier (keep full line)
    sig, mu = efficient_frontier_unconstrained(means, covmat, n=400)
    if sig.size and mu.size:
        ax.plot(sig, mu, color=frontier_color, linewidth=2.8,
                label='Efficient Frontier (IS)', zorder=2)

    # Limits from both points & frontier (no clipping)
    all_x = np.concatenate([pfs['Standard Deviation'].dropna().values, sig])
    all_y = np.concatenate([pfs['Expected Return'].dropna().values,     mu ])
    xpad = (all_x.max() - all_x.min()) * 0.08 if all_x.size else 0.01
    ypad = (all_y.max() - all_y.min()) * 0.08 if all_y.size else 0.01
    ax.set_xlim(all_x.min() - xpad, all_x.max() + xpad)
    ax.set_ylim(all_y.min() - ypad, all_y.max() + ypad)

    ax.set_xlabel('Standard Deviation')
    ax.set_ylabel('Expected Return')
    ax.set_title(title)
    ax.legend(frameon=True, loc='best')
    ax.grid(True, linestyle=':', alpha=0.7)

    if use_letters and letter_map:
        _add_letter_legend(ax, letter_map, title="Letter → Portfolio", loc=letter_legend_loc)

    plt.tight_layout()
    return fig, ax, letter_map

def plot_is_os_lw_frontier(
    pfslw: pd.DataFrame, means: pd.Series, lw_covmat: pd.DataFrame, rfrate: float, title: str,
    color_is='#1f77b4', color_os='#ff7f0e', cal_color='#2ca02c', frontier_color='#17becf',
    use_letters: bool = True, letter_legend_loc: str = "lower right"
):
    for col in ['Standard Deviation', 'Expected Return']:
        pfslw[col] = pd.to_numeric(pfslw[col], errors='coerce')

    idx_norm = pfslw.index.astype(str).str.lower().str.strip()
    is_mask = idx_norm.str.startswith('is ')
    os_mask = idx_norm.str.startswith('os ')
    letter_map = _letter_map_from_index(pfslw.index.tolist()) if use_letters else {}

    sns.set(style='whitegrid')
    fig, ax = plt.subplots(figsize=(8.5,6.5), dpi=160)

    if pfslw.loc[is_mask].size:
        sub = pfslw.loc[is_mask, ['Standard Deviation','Expected Return']].dropna()
        ax.scatter(sub['Standard Deviation'], sub['Expected Return'],
                   c=color_is, s=110, label='IS', zorder=4,
                   edgecolors="white", linewidths=1.2)
        _annotate_static(ax, pfslw, is_mask, color_is, letter_map, dy_pixels=12)

    if pfslw.loc[os_mask].size:
        sub = pfslw.loc[os_mask, ['Standard Deviation','Expected Return']].dropna()
        ax.scatter(sub['Standard Deviation'], sub['Expected Return'],
                   c=color_os, s=130, marker='^', label='OOS', zorder=4,
                   edgecolors="white", linewidths=1.2)
        _annotate_static(ax, pfslw, os_mask, color_os, letter_map, dy_pixels=-14)

    # CAL via "IS LW QP ORP"
    tag = 'is lw qp orp'
    poss = idx_norm[idx_norm.str.contains(tag)].tolist()
    if poss:
        label = pfslw.index[idx_norm == poss[0]][0]
        sd = float(pfslw.loc[label, 'Standard Deviation']); er = float(pfslw.loc[label, 'Expected Return'])
        if np.isfinite(sd) and np.isfinite(er):
            slope = (er - rfrate)/max(sd, 1e-12)
            x_max = max(float(pfslw['Standard Deviation'].max())*1.25, sd*1.35)
            xs = np.linspace(0, x_max, 400); ys = rfrate + slope*xs
            ax.plot(xs, ys, color=cal_color, linestyle='--', linewidth=2.4,
                    label=f'CAL via {label}', zorder=2.5)
            ax.scatter([0],[rfrate], color=cal_color, edgecolor='k', zorder=5)

    # Efficient frontier (LW; keep full line)
    sig, mu = efficient_frontier_unconstrained(means, lw_covmat, n=400)
    if sig.size and mu.size:
        ax.plot(sig, mu, color=frontier_color, linewidth=2.8,
                label='Efficient Frontier (IS, LW)', zorder=2)

    all_x = np.concatenate([pfslw['Standard Deviation'].dropna().values, sig])
    all_y = np.concatenate([pfslw['Expected Return'].dropna().values,     mu ])
    xpad = (all_x.max() - all_x.min()) * 0.08 if all_x.size else 0.01
    ypad = (all_y.max() - all_y.min()) * 0.08 if all_y.size else 0.01
    ax.set_xlim(all_x.min() - xpad, all_x.max() + xpad)
    ax.set_ylim(all_y.min() - ypad, all_y.max() + ypad)

    ax.set_xlabel('Standard Deviation')
    ax.set_ylabel('Expected Return')
    ax.set_title(title)
    ax.legend(frameon=True, loc='best')
    ax.grid(True, linestyle=':', alpha=0.7)

    if use_letters and letter_map:
        _add_letter_legend(ax, letter_map, title="Letter → Portfolio", loc=letter_legend_loc)

    plt.tight_layout()
    return fig, ax, letter_map

# ---------- SAVE ----------
# %matplotlib inline  # (Jupyter)

# 1) SAMPLE Σ
fig, ax, letter_map_sample = plot_is_os_frontier(
    pfs, means, covmat, rfrate,
    title="Sample Covariance: IS and OOS Points",
    use_letters=True
)
# plt.show()
fig.savefig("outputs/samplecovmat_graph.png", dpi=180, bbox_inches="tight"); plt.close(fig)

# 2) LEDOIT–WOLF Σ
fig2, ax2, letter_map_lw = plot_is_os_lw_frontier(
    pfslw, means, lw_covmat, rfrate,
    title="Ledoit–Wolf Covariance: IS and OOS Points",
    use_letters=True
)
# plt.show()
fig2.savefig("outputs/lwcovmat_graph.png", dpi=180, bbox_inches="tight"); plt.close(fig2)
plt.close(fig)
plt.close(fig2)

/tmp/ipython-input-250749323.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = float(one.T @ P @ one); b = float(one.T @ P @ mu); c = float(mu.T @ P @ mu)
/tmp/ipython-input-250749323.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = float(one.T @ P @ one); b = float(one.T @ P @ mu); c = float(mu.T @ P @ mu)


In [ ]:
# ============ Excel Report Builder (clean, percent-safe) ============
from pathlib import Path
import pandas as pd
import numpy as np
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill
from openpyxl.drawing.image import Image as XLImage

# ---------- helpers ----------
def _autosize(ws, max_width=48):
    for col in ws.columns:
        col_letter = get_column_letter(col[0].column)
        width = 0
        for cell in col:
            txt = "" if cell.value is None else str(cell.value)
            width = max(width, len(txt))
        ws.column_dimensions[col_letter].width = min(max(10, width + 2), max_width)

def _write_df(ws, df: pd.DataFrame, start_row=1, start_col=1, header=True, index_as_col=True):
    r, c = start_row, start_col
    if header:
        if index_as_col:
            ws.cell(r, c).value = ""
            ws.cell(r, c).font = Font(bold=True)
            for j, col in enumerate(df.columns, start=c+1):
                ws.cell(r, j).value = str(col)
                ws.cell(r, j).font = Font(bold=True)
        else:
            for j, col in enumerate(df.columns, start=c):
                ws.cell(r, j).value = str(col)
                ws.cell(r, j).font = Font(bold=True)
        r += 1
    for idx, row in df.iterrows():
        if index_as_col:
            ws.cell(r, c).value = str(idx)
            ws.cell(r, c).font = Font(bold=True)
            for j, col in enumerate(df.columns, start=c+1):
                val = row[col]
                ws.cell(r, j).value = float(val) if pd.api.types.is_number(val) else (None if pd.isna(val) else str(val))
        else:
            for j, col in enumerate(df.columns, start=c):
                val = row[col]
                ws.cell(r, j).value = float(val) if pd.api.types.is_number(val) else (None if pd.isna(val) else str(val))
        r += 1
    return r

def _format_percent(ws, header_row=1, include_cols=None, exclude_cols=None):
    # exact matches are case-insensitive
    headers = {}
    for cell in ws[header_row]:
        if cell.value:
            headers[cell.column] = str(cell.value).strip()
    norm = lambda s: s.lower().strip()
    include = {norm(c) for c in include_cols} if include_cols else set()
    exclude = {norm(c) for c in exclude_cols} if exclude_cols else set()
    keywords = ["expected return", "standard deviation", "return", "ret", "stdev", "std dev", "vol", "volatility"]
    for ci, name in headers.items():
        n = norm(name)
        if n in exclude:
            continue
        use_pct = (n in include) if include else (any(k in n for k in keywords) or n.endswith("(ann.)") or n.endswith("_a") or n.endswith("_m"))
        if not use_pct:
            continue
        col_letter = get_column_letter(ci)
        for cell in ws[col_letter][header_row:]:
            if isinstance(cell.value, (int, float)):
                cell.number_format = "0.00%"

def _format_ratio(ws, header_row=1):
    headers = {}
    for cell in ws[header_row]:
        if cell.value:
            headers[cell.column] = str(cell.value).lower()
    for ci, name in headers.items():
        if "sharpe" in name or "ratio" in name:
            col_letter = get_column_letter(ci)
            for cell in ws[get_column_letter(ci)][header_row:]:
                if isinstance(cell.value, (int, float)):
                    cell.number_format = "0.000"

def _freeze(ws, row=2, col=2):
    ws.freeze_panes = ws.cell(row=row, column=col)

def _stripe(ws, header_row=1):
    fill = PatternFill(start_color="FFF5F5F5", end_color="FFF5F5F5", fill_type="solid")
    on = False
    for r in range(header_row+1, ws.max_row+1):
        on = not on
        if on:
            for c in range(1, ws.max_column+1):
                ws.cell(r, c).fill = fill

# ---------- main builder ----------
def build_excel_report(
    path_xlsx: str,
    inputs: dict,
    pfs: pd.DataFrame,      # sample-cov metrics (rows like IS ..., OS ...)
    pfslw: pd.DataFrame,    # LW metrics (rows like IS ..., OS ...)
    is_returns: pd.DataFrame,
    os_returns: pd.DataFrame,
    is_covrep: pd.DataFrame = None,
    os_covrep: pd.DataFrame = None,
    image_paths: list = None
):
    wb = Workbook()

    # Inputs
    ws = wb.active
    ws.title = "Inputs"
    ws["A1"].value = "Parameter"; ws["A1"].font = Font(bold=True)
    ws["B1"].value = "Value";     ws["B1"].font = Font(bold=True)
    r = 2
    for k, v in inputs.items():
        ws[f"A{r}"] = str(k)
        ws[f"B{r}"] = str(v)
        r += 1
    _autosize(ws); _freeze(ws)

    # IS Metrics (sample Σ)
    ws_is = wb.create_sheet("IS Metrics")
    _write_df(ws_is, pfs.loc[pfs.index.str.startswith("IS")], header=True, index_as_col=True)
    _format_percent(ws_is, header_row=1,
                    include_cols=["Expected Return","Standard Deviation","Expected Return (Ann.)","Standard Deviation (Ann.)"],
                    exclude_cols=["Sharpe Ratio (Monthly)","Sharpe Ratio (Annualized)"])
    _format_ratio(ws_is, header_row=1)
    _stripe(ws_is); _autosize(ws_is); _freeze(ws_is)

    # OS Metrics (sample Σ)
    ws_os = wb.create_sheet("OS Metrics")
    _write_df(ws_os, pfs.loc[pfs.index.str.startswith("OS")], header=True, index_as_col=True)
    _format_percent(ws_os, header_row=1,
                    include_cols=["Expected Return","Standard Deviation","Expected Return (Ann.)","Standard Deviation (Ann.)"],
                    exclude_cols=["Sharpe Ratio (Monthly)","Sharpe Ratio (Annualized)"])
    _format_ratio(ws_os, header_row=1)
    _stripe(ws_os); _autosize(ws_os); _freeze(ws_os)

    # IS–LW Metrics
    ws_is_lw = wb.create_sheet("IS–LW Metrics")
    _write_df(ws_is_lw, pfslw.loc[pfslw.index.str.startswith("IS")], header=True, index_as_col=True)
    _format_percent(ws_is_lw, header_row=1,
                    include_cols=["Expected Return","Standard Deviation","Expected Return (Ann.)","Standard Deviation (Ann.)"],
                    exclude_cols=["Sharpe Ratio (Monthly)","Sharpe Ratio (Annualized)"])
    _format_ratio(ws_is_lw, header_row=1)
    _stripe(ws_is_lw); _autosize(ws_is_lw); _freeze(ws_is_lw)

    # OS–LW Metrics
    ws_os_lw = wb.create_sheet("OS–LW Metrics")
    _write_df(ws_os_lw, pfslw.loc[pfslw.index.str.startswith("OS")], header=True, index_as_col=True)
    _format_percent(ws_os_lw, header_row=1,
                    include_cols=["Expected Return","Standard Deviation","Expected Return (Ann.)","Standard Deviation (Ann.)"],
                    exclude_cols=["Sharpe Ratio (Monthly)","Sharpe Ratio (Annualized)"])
    _format_ratio(ws_os_lw, header_row=1)
    _stripe(ws_os_lw); _autosize(ws_os_lw); _freeze(ws_os_lw)

    # Monthly Returns (IS)
    ws_ris = wb.create_sheet("Monthly Returns (IS)")
    df_is = is_returns.copy()
    df_is.index = pd.to_datetime(df_is.index)
    table = df_is.sort_index().reset_index()
    table.rename(columns={table.columns[0]: "Date"}, inplace=True)
    table["Date"] = pd.to_datetime(table["Date"]).dt.strftime("%Y-%m")
    _write_df(ws_ris, table, header=True, index_as_col=False)
    for col in range(2, ws_ris.max_column+1):
        for cell in ws_ris[get_column_letter(col)][1:]:
            if isinstance(cell.value, (int, float)):
                cell.number_format = "0.00%"
    _autosize(ws_ris); _freeze(ws_ris)

    # Monthly Returns (OS)
    ws_ros = wb.create_sheet("Monthly Returns (OS)")
    df_os = os_returns.copy()
    df_os.index = pd.to_datetime(df_os.index)
    table = df_os.sort_index().reset_index()
    table.rename(columns={table.columns[0]: "Date"}, inplace=True)
    table["Date"] = pd.to_datetime(table["Date"]).dt.strftime("%Y-%m")
    _write_df(ws_ros, table, header=True, index_as_col=False)
    for col in range(2, ws_ros.max_column+1):
        for cell in ws_ros[get_column_letter(col)][1:]:
            if isinstance(cell.value, (int, float)):
                cell.number_format = "0.00%"
    _autosize(ws_ros); _freeze(ws_ros)

    # Coverage (optional)
    if is_covrep is not None:
        ws_cov_is = wb.create_sheet("Coverage (IS)")
        _write_df(ws_cov_is, is_covrep, header=True, index_as_col=True)
        _format_percent(ws_cov_is, header_row=1)  # if your frac column is 0..1
        _stripe(ws_cov_is); _autosize(ws_cov_is); _freeze(ws_cov_is)

    if os_covrep is not None:
        ws_cov_os = wb.create_sheet("Coverage (OS)")
        _write_df(ws_cov_os, os_covrep, header=True, index_as_col=True)
        _format_percent(ws_cov_os, header_row=1)
        _stripe(ws_cov_os); _autosize(ws_cov_os); _freeze(ws_cov_os)

    # Plots
    ws_plots = wb.create_sheet("Plots")
    cur_row = 1
    if image_paths:
        for p in image_paths:
            try:
                img = XLImage(p)
                img.anchor = f"A{cur_row}"
                ws_plots.add_image(img)
                advance = max(int(img.height / 18) + 2, 20)
                cur_row += advance
            except Exception as e:
                ws_plots.cell(cur_row, 1).value = f"Could not embed {p}: {e}"
                cur_row += 2
    _autosize(ws_plots)

    Path(path_xlsx).parent.mkdir(parents=True, exist_ok=True)
    wb.save(path_xlsx)
    print(f"[Excel] Saved {path_xlsx}")
    return path_xlsx


In [ ]:
# ---- Make sure the images you embed are the ones you saved ----
# You saved to: outputs/samplecovmat_graph.png and outputs/lwcovmat_graph.png
# so use those here (and feel free to add more later).
from pathlib import Path
outdir = Path("outputs")
outdir.mkdir(exist_ok=True)

image_paths = [
    str(outdir / "samplecovmat_graph.png"),
    str(outdir / "lwcovmat_graph.png"),
]

# ---- Optional but nice: build weights tables for Excel ----
# Sample-Σ weights
weights_sample = (
    pd.DataFrame({
        "Original":      weights.reindex(ret_matrix.columns),
        "GMV":           gmv.reindex(ret_matrix.columns),
        "Efficient":     efport.reindex(ret_matrix.columns),
        "ORP":           orp.reindex(ret_matrix.columns),
        "QP ORP":        qporp.reindex(ret_matrix.columns)
    })
    .rename_axis("Ticker")
)

# Ledoit–Wolf-Σ weights
weights_lw = (
    pd.DataFrame({
        "LW GMV":        lwgmv.reindex(ret_matrix.columns),
        "LW Efficient":  lwefport.reindex(ret_matrix.columns),
        "LW ORP":        lworp.reindex(ret_matrix.columns),
        "LW QP ORP":     lwqporp.reindex(ret_matrix.columns)
    })
    .rename_axis("Ticker")
)

# Differences (LW minus Sample where comparable)
# Only compare columns with the same semantics
overlap_cols = ["GMV", "Efficient", "ORP", "QP ORP"]
diff_cols = {f"Δ {c} (LW - Sample)": (weights_lw.get("LW "+c) - weights_sample.get(c))
             for c in overlap_cols if ("LW "+c) in weights_lw.columns and c in weights_sample.columns}
weights_diff = pd.DataFrame(diff_cols).rename_axis("Ticker")

# ---- Annualize the metrics (optional lines to make Excel nicer to read) ----
def _annualize(df):
    df = df.copy()
    if "Expected Return" in df.columns:
        df["Expected Return (Ann.)"] = df["Expected Return"] * 12
    if "Standard Deviation" in df.columns:
        df["Standard Deviation (Ann.)"] = df["Standard Deviation"] * (12 ** 0.5)
    return df

pfs_out   = _annualize(pfs)
pfslw_out = _annualize(pfslw)

# ---- Inputs panel (shown on first sheet) ----
inputs = {
    "Tickers": ", ".join(list(ret_matrix.columns)),
    "IS Range": f"{ret_matrix.index.min():%Y-%m} – {ret_matrix.index.max():%Y-%m}",
    "OOS Range": f"{os_ret_matrix.index.min():%Y-%m} – {os_ret_matrix.index.max():%Y-%m}",
    "Risk-free (monthly)": f"{rfrate:.6f}",
    "Constraints": "Unconstrained (weights sum to 1) unless stated for QP",
    "QP bounds": "[-1, 1] in this run (edit in code)",
}

# ---- Build the Excel workbook (adds metrics, returns, and plots you specify) ----
# NOTE: build_excel_report is already defined in your file.
# We’ll extend it here by writing the extra weight sheets after it saves.
xlsx_path = outdir / "Project2_Report.xlsx"

build_excel_report(
    path_xlsx=str(xlsx_path),
    inputs=inputs,
    pfs=pfs_out,          # Sample-Σ metrics (IS* and OOS*)
    pfslw=pfslw_out,      # LW-Σ metrics (IS* and OOS*)
    is_returns=ret_matrix,
    os_returns=os_ret_matrix,
    is_covrep=None,
    os_covrep=None,
    image_paths=image_paths
)

# ---- Append weight sheets to the same workbook ----
from openpyxl import load_workbook
wb = load_workbook(str(xlsx_path))

def _append_df_sheet(wb, title, df, pct_cols=None):
    ws = wb.create_sheet(title)
    # write header
    ws.cell(1,1).value = "Ticker"; ws.cell(1,1).font = Font(bold=True)
    for j, c in enumerate(df.columns, start=2):
        ws.cell(1, j).value = c; ws.cell(1, j).font = Font(bold=True)
    # write rows
    r = 2
    for idx, row in df.iterrows():
        ws.cell(r,1).value = str(idx)
        for j, c in enumerate(df.columns, start=2):
            val = row[c]
            ws.cell(r, j).value = float(val) if pd.notna(val) else None
        r += 1
    # formatting
    # treat all weight columns as percents
    from openpyxl.styles import numbers
    for col in range(2, 2 + len(df.columns)):
        for rr in range(2, r):
            cell = ws.cell(rr, col)
            if isinstance(cell.value, (int, float)):
                cell.number_format = "0.00%"
    # tidy
    _autosize(ws); _freeze(ws, row=2, col=2); _stripe(ws)

_append_df_sheet(wb, "Weights (Sample Σ)", weights_sample)
_append_df_sheet(wb, "Weights (LW Σ)",     weights_lw)
_append_df_sheet(wb, "Weights Δ (LW-Sample)", weights_diff)

wb.save(str(xlsx_path))
print(f"[Excel] Appended weight sheets → {xlsx_path}")


[Excel] Saved outputs/Project2_Report.xlsx
[Excel] Appended weight sheets → outputs/Project2_Report.xlsx
